Author:
        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/4956778/8344638/a2a6aa289fce8461958dc287f1dab799/dataset-cover.jpg?t=2024-05-07-09-36-53" alt="IMG">
</p>

<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 1. Content</i></h1>

<br>

> <h4 style = 'font-family: Times New Roman'>
This dataset explores how weather conditions impact renewable energy generation. <br><br>Spanning from January 1, 2017, to August 31, 2022, the dataset provides climate data such as temperature, pressure, wind speed, and sunlight duration at 15-minute intervals. <br><br>By incorporating variables like GHI and SunlightTime, it enables the prediction of solar energy production.


- DataSource: https://www.kaggle.com/datasets/pythonafroz/renewable-power-generation-and-weather-conditions/data
  
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 2. About Dataset</i></h1>

<br> 

<details>
    <summary>More Columns Info</summary>
    17 columns in the dataset, below is <u>description of main features</u>:

    *  (1) Time: The timestamp of the recorded data in the format of YYYY-MM-DD HH:MM:SS.
        
    *  (2) Energy delta[Wh]: The difference in energy consumption in Watt-hours (Wh) from the previous timestamp to the current timestamp.
        
    *  (3) GHI: Global Horizontal Irradiance in Watts per square meter (W/m²) measured by a pyranometer.
        
    *  (4) temp: The temperature in degrees Celsius (°C) measured at the same height as the pyranometer.

    *  (5) pressure: The atmospheric pressure in hectopascals (hPa) measured at the same height as the pyranometer.

    *  (6) humidity: The relative humidity in percentage (%) measured at the same height as the pyranometer.

    *  (7) wind_speed: The wind speed in meters per second (m/s) measured at the same height as the pyranometer.

    *  (8) rain_1h: The amount of precipitation in millimeters (mm) measured over the past hour.
    
    *  (9) snow_1h: The amount of snowfall in millimeters.

    *  (10) clouds_all: The cloud situation.
</details>    
    
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 3. Goal of the Notebook</i></h1>
    
> <h4 style = 'font-family: Times New Roman'>
Goal is to train and test model using GPT, Pytorch. <br><br>
    The Target Column used here is <b>Energy delta[Wh]</b>
</h4>

 

***

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import pandas  as pd
df = pd.read_csv(path_append + '../data/Renewable Power Generation and weather Conditions/Renewable.csv')
df.head()

,Time,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month
0,2017-01-01 00:00:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
1,2017-01-01 00:15:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
2,2017-01-01 00:30:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
3,2017-01-01 00:45:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
4,2017-01-01 01:00:00,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1


<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [3]:
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')

# Extract time in seconds from the datetime object
df['SecondsOfDay'] = df['Time'].dt.hour * 3600 + df['Time'].dt.minute * 60 + df['Time'].dt.second

# Optionally drop the original Time column
df.drop(['Time'], axis=1, inplace=True)

# Display the first few rows to verify the changes
df.head()

,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month,SecondsOfDay
0,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,0
1,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,900
2,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,1800
3,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,2700
4,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1,3600


In [4]:
from tools.preprocessing.data_frame import process_dataframe, preprocess_cyclical_columns
target_columns = ['Energy delta[Wh]']
one_hot_columns = ['weather_type']
df = preprocess_cyclical_columns(df, ['month'])
df, description = process_dataframe(df, target_columns, one_hot_columns = one_hot_columns) 

description

,Min,Max,Mean,Std,Null Count
GHI,-0.034188,4.863248,6.623192e-01,1.114787,0
temp,-4.213083,5.929421,8.950458e-01,1.547589,0
pressure,0.924568,1.062459,1.000000e+00,0.018883,0
humidity,-2.818182,0.727273,-1.904288e-01,0.709294,0
wind_speed,-1.541667,4.416667,9.906103e-02,0.759039,0
rain_1h,-0.066035,8.023965,1.141051e-17,0.278913,0
snow_1h,-0.007148,2.812852,-1.306702e-18,0.069710,0
clouds_all,-1.801176,0.928938,-1.850958e-16,1.000003,0
isSun,-0.519962,0.480038,-9.009250e-18,0.499603,0
sunlightTime,-0.076923,2.538462,4.659515e-01,0.702313,0


{'num_features': 21,
 'num_classes': 1,
 'encoded_columns': Index([], dtype='object'),
 'target_encoded_columns': Index([], dtype='object'),
 'scalers': {'GHI': 'robust',
  'SecondsOfDay': 'minmax',
  'SunlightTime/daylength': 'none',
  'clouds_all': 'standard',
  'dayLength': 'minmax',
  'hour': 'minmax',
  'humidity': 'robust',
  'isSun': 'none',
  'month_cos': 'none',
  'month_sin': 'none',
  'pressure': 'minmax',
  'rain_1h': 'none',
  'snow_1h': 'none',
  'sunlightTime': 'robust',
  'temp': 'minmax',
  'wind_speed': 'robust'}}

In [5]:
df.min(), df.max() 

(GHI                      -0.034188
 temp                     -4.213083
 pressure                  0.924568
 humidity                 -2.818182
 wind_speed               -1.541667
 rain_1h                  -0.066035
 snow_1h                  -0.007148
 clouds_all               -1.801176
 isSun                    -0.519962
 sunlightTime             -0.076923
 dayLength                 0.202173
 SunlightTime/daylength   -0.265187
 hour                     -1.000000
 SecondsOfDay             -1.000000
 month_sin                -1.022387
 month_cos                -1.005086
 weather_type_1            0.000000
 weather_type_2            0.000000
 weather_type_3            0.000000
 weather_type_4            0.000000
 weather_type_5            0.000000
 Energy delta[Wh]          0.000000
 dtype: float64,
 GHI                          4.863248
 temp                         5.929421
 pressure                     1.062459
 humidity                     0.727273
 wind_speed                   4.416

In [6]:
import torch
from sklearn.model_selection import train_test_split
from tools.preprocessing.template_dataset import TemplateDataset

max_seq_len = 32
min_seq_len = 16
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, shuffle=False)
# predict the next value in the sequence
train_df_x = train_df.iloc[:, :-1] # all columns except the last one
train_df_y = train_df.iloc[:, -1:] # only the last column

test_df_x = test_df.iloc[:, :-1] # all columns except the last one
test_df_y = test_df.iloc[:, -1:] # only the last column

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
trainset = TemplateDataset(train_df_x, train_df_y, min_seq_len, max_seq_len)
testset = TemplateDataset(test_df_x, test_df_y, min_seq_len, max_seq_len)

train df shape:  (137743, 22)
test df shape:  (59033, 22)


<a id="3"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #427bf5'>3.</b> Modeling</b></h1>

In [7]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

num_features = description['num_features']
num_classes = description['num_classes']
data_config = DataConfig(dataset_name = 'renewable-power-gen-prediction', task_type='regression', obs_shape=[num_features], label_size=num_classes)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'gpt', encoder_network = 'none')
ml_params.algorithm.error_function = 'mae'

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=True) 

In [8]:
trainer_hub.train(trainset, testset)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junhopark (ccnets). Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\..\saved\renewable-power-gen-prediction\causal-learning)... Done. 0.0s


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2151 [00:00<?, ?it/s]

[0/100][100/2151][Time 10.34]
Unified LR across all optimizers: 0.0001993957766378747
--------------------Training Metrics--------------------
CCNet:  Three Gpt
Inf: 0.1128	Gen: 0.2016	Rec: 0.2219	E: 0.0921	R: 0.1316	P: 0.3116
--------------------Test Metrics------------------------
mse: 1950621.1250
mae: 737.2778
r2: -0.3755

[0/100][200/2151][Time 9.91]
Unified LR across all optimizers: 0.00019879933411171295
--------------------Training Metrics--------------------
CCNet:  Three Gpt
Inf: 0.0297	Gen: 0.0901	Rec: 0.0928	E: 0.0267	R: 0.0319	P: 0.1552
--------------------Test Metrics------------------------
mse: 1606258.8750
mae: 600.0421
r2: -0.2774

[0/100][300/2151][Time 9.92]
Unified LR across all optimizers: 0.00019820467569398644
--------------------Training Metrics--------------------
CCNet:  Three Gpt
Inf: 0.0204	Gen: 0.0770	Rec: 0.0784	E: 0.0190	R: 0.0217	P: 0.1358
--------------------Test Metrics------------------------
mse: 1784630.7500
mae: 708.8958
r2: -0.3782

[0/100][400/2

In [ ]:
trainer_hub.test(testset)